In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval

In [ ]:
df = pd.read_csv('./movie_dataset.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
df['budget'] = df['budget'].replace({0: np.NAN})
df.dropna(subset=['budget', 'mpaa', 'director', 'genres', 'domestic_distributor', 'production_companies', 'production_countries', 'spoken_languages'], inplace=True) 

df = df[df['status'] == 'Released']
df = df.drop(columns=['status'], axis=1)

In [ ]:
cols = ['genres', 'production_countries', 'production_companies', 'actors', 'spoken_languages']

mlb_classes = {}
for col in cols:
    df[col] = df.apply(lambda x: literal_eval(x[col]), axis=1)

    mlb = MultiLabelBinarizer()
    mlb.fit_transform(df[col])
    mlb_classes[col] = mlb.classes_
    print(col, len(mlb.classes_))

In [ ]:
mlb = MultiLabelBinarizer()
dum = mlb.fit_transform(df['genres'])
classes = ['genre_' + x.replace(' ', '_').lower() for x in mlb.classes_]
df = df.join(pd.DataFrame(dum.astype(bool), df.index, classes))

In [ ]:
genres = set(mlb.classes_)
print('Количество жанров:', len(genres))
print('Жанры: ', genres)

In [ ]:
df = df.drop(columns=['genres'], axis=1)

In [ ]:
df = df.join(pd.DataFrame(df['actors'].values.tolist(), df.index, ['actor_1', 'actor_2', 'actor_3']))
df = df.drop(['actors'], axis=1)

TODO:
1. Попробовать one-hot encoding для production_countries, production_companies и spoken_languages; target encoding для actors
2. Попробовать для все вышеперечисленных столбцов target encoding

Сначала применим one-hot encoding для полей `production_countries`, `production_companies` и `spoken_languages`

In [ ]:
b = df['production_companies'].explode().value_counts()

In [ ]:
a[a['count'] > 20].shape

In [ ]:
df.apply(lambda x: x.mask(x.map(b.value_counts()) < 10, 'other') if x.name == 'production_companies' else x)

In [ ]:
# def dict_list_to_df(s, col):
#     rows = []
#     for index, row in s.items():
#         for item in row:
#             rows.append(item)
#     df = pd.DataFrame(rows)
#     return df


# df_genres = df['genres'].apply(lambda i: literal_eval(i))
# df_genres = dict_list_to_df(df_genres, 'genres')
# df_unique_genres = df_genres.drop_duplicates()
# df_unique_genres.set_index('id', inplace=True)